### Featurizing KLM data for date predictions
This notebook will be the featurizing pipeline for the model which will be used to make predictions in regards to whether a user will convert in a given time-window.
Specifically, the model is dealing only with conversions events, which means that all other events are not taken into account when making predictions with this model.

For the model to work properly, the relevant fields which the model will look at are the origin, destination, travelClass, LOS, DBD, pax and the value of the deal. Based on those feautres, the model will be able to classify the users in the dataset with either 1 or 0 for a given time-window.

#### Keep in mind:
The time-window can be changed by rewriting the `target_formatter` (SQLTransformer) in order to give predictions over other time-windows.
For the sake of evaluation, the model will make predictions over the time-window of `current_date - 160 days` in order to fit the current data present, so the model will try to predict which users will book on this time-window. 

In [1]:
%%cleanup -f

In [2]:
%%configure -f
{"driverMemory": "48G", "executorMemory": "8G", "executorCores": 2, "numExecutors": 50}

In [3]:
%%info

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("date_predictions_model") \
    .config("spark.driver.maxResultSize", "10g") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,None,pyspark3,idle,,,✔


SparkSession available as 'spark'.


In [5]:
spark.sql("set spark.sql.caseSensitive=true")

DataFrame[key: string, value: string]

In [6]:
from pyspark.sql import SQLContext
sqlC = SQLContext(sc)

In [7]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import *
from pyspark.ml.feature import SQLTransformer
from pyspark.ml import Pipeline

#### s3 location where featurized pipeline will be written to

In [8]:
directory = 's3://r42-data-scientists/tbuaron/klm'
model_dir = 'booking_dates'
pipeline_name = 'featurizing_pipeline'

#### s3 location to where the parquet data is currently in

In [9]:
parquet_files = 's3://r42-analysis-adhoc/klm-export-parquet/201803*'
ddf = spark.read.parquet(parquet_files)

#### s3 location to where the columns mapping dictionary is in

In [10]:
dict_loc = ('%s/%s/date_predictions_dict.json' % (directory, model_dir))
column_map = spark.read.json(dict_loc)

### list of transformers to use

#### renaming columns transformer

In [11]:
columns_transformer = SQLTransformer(statement='SELECT timestamp, CAST(trackId as string) AS user_id, CAST(interactionType as string) \
                                                AS interaction_type, CAST(value as int) AS item_value, \
                                                CAST(conversion_variables_destination as string) AS item_destination_conv, \
                                                CAST(conversion_variables_country as string) AS item_id_conv, \
                                                CAST(conversion_variables_origin as string) AS item_origin_conv, \
                                                CAST(conversion_variables_travelClass as string) AS item_travel_class_conv, \
                                                CAST(conversion_variables_tripCode as string) AS user_tripCode, \
                                                CAST(conversion_variables_booking_timestamp as date) AS item_booking_timestamp_conv, \
                                                CAST(conversion_variables_dd as date) AS item_departure_date_1_conv, \
                                                CAST(conversion_variables_rd as date) AS item_return_date_1_conv, \
                                                CAST(conversion_variables_returnDate as date) as item_return_date_2_conv, \
                                                CAST(conversion_variables_pax as int) as item_pax_conv FROM __THIS__')

#### Conversion transformer

In [12]:
conversions_transformer = SQLTransformer(statement="SELECT * FROM __THIS__ WHERE interaction_type == 'conversion'")

#### fill null values

In [13]:
null_formatter = SQLTransformer(statement="SELECT *, nullif(item_origin_conv, '') as conversion_origin, nullif(item_destination_conv, '') as conversion_destination, nullif(item_pax_conv, '') as pax_cleaned FROM __THIS__")

#### pax transformer

In [14]:
pax_transformer = SQLTransformer(statement='SELECT *, CASE WHEN pax_cleaned IS NULL THEN 0 ELSE pax_cleaned END AS pax_ FROM __THIS__')

#### cast columns transformer
keep in mind: this transformer cast `two` return dates, for other clients it is possible that we will need to remove one of them because other clients might have only one return date

In [15]:
casting_transformer = SQLTransformer(statement="SELECT *, CAST(item_booking_timestamp_conv AS date) AS booking_timestamp, CAST(item_departure_date_1_conv AS date) AS departure_date, CAST(item_return_date_1_conv AS date) AS returnDate1, CAST(item_return_date_2_conv AS date) AS returnDate2, CAST(pax_ AS int) AS pax FROM __THIS__")

#### return date transformer
###### KLM specific transformer
In KLM data, there are two columns related to the `return_date` of a user, in some cases `return_date_1` will be present, in other cases `return_date_2` will be present. Therefore, this transformer is aimed on joining those two columns together whenever the `return_date` column is present and not None.

In [16]:
return_date_formatter = SQLTransformer(statement='SELECT *, CASE WHEN returnDate1 is not NULL THEN SUBSTRING(returnDate1, 1, 10) WHEN returnDate2 is not NULL THEN SUBSTRING(returnDate2, 1, 10) END AS return_date FROM __THIS__')

#### origin transformer

In [17]:
origin_formatter = SQLTransformer(statement="SELECT *, CASE WHEN conversion_origin is not NULL THEN SUBSTRING(conversion_origin, 1, 3) ELSE 'UND' END AS origin FROM __THIS__")

#### destination transformer

In [18]:
destination_formatter = SQLTransformer(statement="SELECT *, CASE WHEN conversion_destination is not NULL THEN SUBSTRING(conversion_destination, 1, 3) ELSE 'UND' END AS destination FROM __THIS__")

#### class transformer
##### KLM specific transformer. 
This transformer checks if a user is booked a business class ticket or a economy class ticket.  This transformer will need to be changed to another transformer which fits the way other clients decide on their travel class

In [19]:
class_formatter = SQLTransformer(statement="SELECT *, CASE WHEN SUBSTRING(item_travel_class_conv, 1, 1) IN ('J', 'C', 'D', 'I', 'Z', 'O') THEN 'business' ELSE 'economy' END AS class FROM __THIS__")

#### LOS transformer

In [20]:
los_formatter = SQLTransformer(statement="SELECT *, CASE WHEN return_date IS NULL THEN 1 ELSE DATEDIFF(return_date, departure_date) END AS LOS FROM __THIS__")

#### DBD transformer

In [21]:
dbd_formatter = SQLTransformer(statement="SELECT *, CASE WHEN departure_date IS NULL THEN 1 ELSE DATEDIFF(departure_date, booking_timestamp) END AS DBD FROM __THIS__")

#### value transformer
price value of the tickets bought

In [22]:
value_transformer = SQLTransformer(statement='SELECT *, CASE WHEN item_value is not NULL THEN item_value ELSE 0 END as price_value FROM __THIS__')

#### booking_type transformer
concatinating `destination` column together with `class` column in order to generate unique `booking_type`

In [23]:
booking_typer = SQLTransformer(statement="SELECT *, CONCAT(destination, '_', class) AS booking_type FROM __THIS__")

#### target transformer 
keeping `-160` for now in order to be able to evaluate the model.
For production, this will need to be changed in orde to fit the latest data

In [24]:
target_formatter = SQLTransformer(statement='SELECT *, CASE WHEN DATEDIFF(DATE_ADD(current_timestamp(), -160), booking_timestamp) BETWEEN -21 AND 0 THEN 1 ELSE 0 END AS label FROM __THIS__')

#### selected columns transformer
keep only relevant columns in the dataframe before fitting it to the pipeline

In [25]:
columns_selector = SQLTransformer(statement='SELECT user_id, timestamp, price_value, booking_timestamp, departure_date, return_date, LOS, DBD, pax, booking_type, origin, label FROM __THIS__')

#### concatenate transformers

In [26]:
all_transformers = [columns_transformer] + [conversions_transformer] + [null_formatter] + [pax_transformer] + [casting_transformer] + [return_date_formatter] + [origin_formatter] + [destination_formatter] + [class_formatter] + [los_formatter] + [dbd_formatter] + [value_transformer] + [booking_typer] + [target_formatter] + [columns_selector]

In [27]:
categorical_features = ['origin', 'booking_type']
indexed_features = [feature + '_idx' for feature in categorical_features]
encoded_features = [feature + '_enc' for feature in indexed_features]
numerical_features = ['pax', 'LOS', 'DBD', 'price_value']
booking_date_features = indexed_features + numerical_features

In [28]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, VectorIndexer

booking_date_indexers = [StringIndexer(inputCol=c, outputCol=c + '_idx', handleInvalid='keep') for c in categorical_features]
booking_date_encoder = OneHotEncoderEstimator(inputCols=indexed_features, outputCols=encoded_features)
booking_date_assembler = VectorAssembler(inputCols=booking_date_features, outputCol='features')

featurizing_pipeline = Pipeline(stages = all_transformers + booking_date_indexers + [booking_date_encoder] \
                                         + [booking_date_assembler])

In [29]:
fitted_featurizing = featurizing_pipeline.fit(ddf)

In [30]:
featurized_ddf = fitted_featurizing.transform(ddf)

#### writing featurized data to s3 takes approximately 1.2 hours 

In [31]:
featurized_ddf.coalesce(1).write.format("parquet").mode("overwrite").save('%s/%s/%s' % (directory, model_dir, pipeline_name), header=True)